Pokémon GAN

By Mohak Jain

In [2]:
import torch
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import argparse
import os
import random
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image,ImageOps,ImageFilter
import cv2

from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Colab Notebooks/data/'


Mounted at /content/drive


# Hyperparams

In [3]:
# https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# Number of workers for dataloader
workers = 2
# Batch size during training
batch_size = 64*3
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64
# Number of channels in the training images.
nc = 5
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 128
# Size of feature maps in discriminator
ndf = 64
# Number of training epochs
num_epochs = 2000


prefix = 'apr4a_'

# Learning rate for optimizers
lr = 0.005
gammaD = 0.5
gammaG = 1
stepsize = 100
dropout = 0.45
useLabelSmoothing = 0.025

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

loadCheckpoint = !False
checkpointToLoad = '1000'


# Setup

In [4]:

def edge_detection(image):
    # Convert PIL image to a numpy array
    numpy_image = np.array(image.convert('L'))  # Convert to grayscale
    # Perform Canny edge detection
    edges = cv2.Canny(numpy_image, threshold1=100, threshold2=200)
    # Dilate the edges to make them more pronounced
    kernel = np.ones((3, 3), np.uint8)
    edges_dilated = cv2.dilate(edges, kernel, iterations=1)

    return edges_dilated


class AddEdgesTransform(object):
    def __call__(self, img):
        # Ensure img is in RGBA
        img_rgba = img.convert('RGBA')
        # Apply edge detection, make sure edges is in 'L' mode for blending
        edges = edge_detection(img_rgba).convert('L')
        # Create a new RGBA image from edges for blending
        edges_rgba = Image.merge('RGBA', (edges, edges, edges, img_rgba.split()[3]))  # Use original alpha
        # Blend the original image with the edges image
        combined = Image.blend(img_rgba, edges_rgba, alpha=0.5)
        return combined


def pil_loader_rgba(path: str) -> torch.Tensor:
    with open(path, 'rb') as f:
        img = Image.open(f).convert('RGBA')
        # Perform edge detection and get the edges as a numpy array
        edges = edge_detection(img)
        # Stack the RGBA channels and edge channel
        img_np = np.array(img)
        img_with_edges_np = np.concatenate((img_np, edges[:, :, None]), axis=2)
        # Convert to PyTorch tensor
        img_with_edges_tensor = torch.tensor(img_with_edges_np.transpose((2, 0, 1)), dtype=torch.float32) / 255.0  # Normalize to [0, 1]
        return img_with_edges_tensor

image_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Normalize((0.5, 0.5, 0.5, 0.5, 0.5), (0.5, 0.5,0.5, 0.5, 0.5))])


dataset = datasets.DatasetFolder(root=data_path, loader=pil_loader_rgba, extensions=['png'], transform=image_transforms)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [5]:
def show_images(image_batch):
    plt.figure(figsize=(8, 8))
    for i, img in enumerate(image_batch):
        ax = plt.subplot(4, 4, i + 1)
        img = img.detach().cpu().numpy()
        img = img.transpose(1, 2, 0)  # Convert to HWC format for visualization

        rgb = img[:, :, :3]

        # Extract the edge channel and normalize
        edges = img[:, :, 4]  # Assumi  ng the 5th channel is edges
        edges = (edges + 1) / 2  # Normalize if necessary

        # Create an edge overlay by repeating the edge channel 3 times to make it RGB
        edges_rgb = np.repeat(edges[:, :, np.newaxis], 3, axis=2)

        # Overlay edges on the RGB image
        # Here, we're simply adding the edge information, you might want to adjust how you combine these
        overlayed = np.clip(rgb + edges_rgb, 0, 1)  # Ensure the combined image is within valid range

        plt.imshow(overlayed)
        plt.axis('off')
    plt.show()

# Fetch a batch of images
images, _ = next(iter(dataloader))
if not loadCheckpoint:
  show_images(images[:16])  # Show first 16 images


In [6]:
import torch.nn as nn
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Nets

In [7]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Dropout(dropout),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.AdaptiveAvgPool2d(1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [8]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# # Print the model
print(netG)

# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# # Print the model
print(netD)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(128, 5, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()


In [9]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

if useLabelSmoothing:
  real_label -= useLabelSmoothing
  fake_label += useLabelSmoothing

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

from torch.optim.lr_scheduler import StepLR
# Setup StepLR learning rate schedulers for both G and D
schedulerD = StepLR(optimizerD, step_size=stepsize, gamma=gammaD)
schedulerG = StepLR(optimizerG, step_size=stepsize, gamma=gammaG)


In [11]:
# Define the path where checkpoints are stored
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/checkpoints/'
if checkpointToLoad == 'latest':
  checkpoint_file = os.path.join(checkpoint_path, 'latest_checkpoint.pth')
  print(checkpoint_file)
else:
  checkpoint_file = os.path.join(checkpoint_path, prefix + 'checkpoint_epoch_' + checkpointToLoad + '.pth')


# Function to load the checkpoint
def load_checkpoint(checkpoint_file, modelG, modelD, optimizerG, optimizerD):
    # Check if file exists
    if os.path.isfile(checkpoint_file):
        print(f"Loading checkpoint '{checkpoint_file}'")
        checkpoint = torch.load(checkpoint_file)
        start_epoch = checkpoint['epoch'] + 1
        modelG.load_state_dict(checkpoint['generator_state_dict'])
        modelD.load_state_dict(checkpoint['discriminator_state_dict'])
        optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
        optimizerD.load_state_dict(checkpoint['optimizerD_state_dict'])
        schedulerG.load_state_dict(checkpoint['schedulerG_state_dict'])
        schedulerD.load_state_dict(checkpoint['schedulerD_state_dict'])
        G_losses = checkpoint.get('G_losses', [])
        D_losses = checkpoint.get('D_losses', [])

        print(f"Loaded checkpoint '{checkpoint_file}' (epoch {checkpoint['epoch']})")
    else:
        start_epoch = 0
        print("No checkpoint found, starting training from scratch.")
    return start_epoch, G_losses, D_losses

start_epoch = 0
D_losses = []
G_losses = []

if loadCheckpoint:
  start_epoch, G_losses, D_losses = load_checkpoint(checkpoint_file, netG, netD, optimizerG, optimizerD)


No checkpoint found, starting training from scratch.


UnboundLocalError: local variable 'G_losses' referenced before assignment

# Training Loop

In [ ]:
img_list = []
iters = 0

print("Starting Training Loop...")

for epoch in range(start_epoch,num_epochs):
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if len(D_losses) > 5 and all(loss > 0.99 for loss in D_losses[-5:]):
          break

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

    # Scheduled learning Rate
    schedulerD.step()
    schedulerG.step()

    # Do checkpoints
    checkpoint = {
        'epoch': epoch,
        'generator_state_dict': netG.state_dict(),
        'discriminator_state_dict': netD.state_dict(),
        'optimizerG_state_dict': optimizerG.state_dict(),
        'optimizerD_state_dict': optimizerD.state_dict(),
        'schedulerG_state_dict': schedulerG.state_dict(),
        'schedulerD_state_dict': schedulerD.state_dict(),
        'lossG': errG.item(),
        'lossD': errD.item(),
        'G_losses': G_losses,
        'D_losses': D_losses,
    }
    torch.save(checkpoint, os.path.join(checkpoint_path, 'latest_checkpoint.pth'))

    if epoch % 25 == 0:
      torch.save(checkpoint, os.path.join(checkpoint_path, f'{prefix}checkpoint_epoch_{epoch}.pth'))



In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))
real_images = real_batch[0][:, :4, :, :]  # Select only RGBA channels, drop the edge channel

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_images.to(device)[:64], padding=5, normalize=True).cpu(), (1, 2, 0)))

with torch.no_grad():
    fake = netG(fixed_noise).detach().cpu()
img_list.append(vutils.make_grid(fake, padding=2, normalize=True))


fake_images = img_list[-1][:4,:,:]
# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(fake_images,(1,2,0)))
plt.show()

In [ ]:
import matplotlib.animation as animation

def create_progress_animation(img_list):
  fig=plt.figure(figsize=(8,8))
  plt.axis('off')
  ims=[]
  for img_tensor in img_list:
    img_rgba = img_tensor[:4,:,:]
    img = np.transpose(img_rgba, (1,2,0))
    im = plt.imshow(img, animated=True)
    ims.append([im])

  ani = animation.ArtistAnimation(fig,ims,interval=500, repeat_delay=1000, blit = True)
  ani.save('/content/drive/MyDrive/Colab Notebooks/'+prefix+"training_progress.gif", writer='imagemagick', dpi=72)
  return ani

ani = create_progress_animation(img_list)